<a href="https://colab.research.google.com/github/aashmauprety/Machine-Learning/blob/master/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
url = 'https://raw.githubusercontent.com/aashmauprety/Machine-Learning/master/Churn_Modelling.csv'

In [0]:
dataset = pd.read_csv(url)

In [0]:
print(dataset)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
#dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
ct = ColumnTransformer([("Country", OneHotEncoder(), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)
X = X[:, 1:]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
# Importing the Keras libraries and packages
# Part 2 - Now let's make the ANN! 

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.layers import Dropout

classifier = Sequential() # Initialising the ANN


classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11)) # Adding the input layer and the first hidden layer
classifier.add(Dropout(p = 0.1)) #add a dropout layer to first hidden layer

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu')) # Adding the second hidden layer
classifier.add(Dropout(p = 0.1)) #add a dropout layer to second hidden layer

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid')) # Adding the output layer


classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # Compiling the ANN


classifier.fit(X_train, y_train, batch_size = 10, epochs = 100) # Fitting the ANN to the Training set


In [0]:
y_predict = classifier.predict(X_test)
y_predict = (y_predict > 0.5)
print(y_predict)

In [0]:
#Cross Validate
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def build_classifier():
 classifier = Sequential() # Initialising the ANN
 classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11)) # Adding the input layer and the first hidden layer
 classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu')) # Adding the second hidden layer
 classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid')) # Adding the output layer
 classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # Compiling the ANN
 #classifier.fit(X_train, y_train, batch_size = 10, epochs = 100) # Fitting the ANN to the Training set
 return classifier

classifier = KerasClassifier(build_fn= build_classifier, batch_size= 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X= X_train, y = y_train, cv = 10, n_jobs= 1)

In [0]:
mean = accuracies.mean()
variance = accuracies.std()
print(mean)
print(variance)

In [0]:
#Parameters Tuning for improving accuracy 
#Using GridSearch to find best combination of hyper paremeters.
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def build_classifier(optimizer):
 classifier = Sequential() # Initialising the ANN
 classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11)) # Adding the input layer and the first hidden layer
 classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu')) # Adding the second hidden layer
 classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid')) # Adding the output layer
 classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy']) # Compiling the ANN
 #classifier.fit(X_train, y_train, batch_size = 10, epochs = 100) # Fitting the ANN to the Training set
 return classifier

classifier = KerasClassifier(build_fn= build_classifier)
parameter = {'batch_size' : [25, 32], 'epochs' : [100, 500], 'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator= classifier, param_grid= parameter, scoring= 'accuracy', cv= 10, verbose= 1)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_scores = grid_search.best_score_